# Building population suitablity layers for GRIDCERF

## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


## 2. Setup environment

### 2.1 Install GDAL

This application requires GDAL to be installed.  We will call GDAL directly from your command prompt or terminal, so please ensure that you can do so before running the following cells.  More information on how to install GDAL can be found here:  https://gdal.org/download.html


### 2.2 Install Python third-party dependencies

The following Python packages can be installed directly from the following kernel if they are not already in your kernel's environment.  If you are using conda, please use conda specific install protocol.

**NOTE**: Executing the following cell will install these Python packages over your existing versions!  


In [ ]:
!pip install -Iv numpy>=1.23.1
!pip install -Iv pandas>=1.4.3
!pip install -Iv rasterio>=1.3.4
!pip install -Iv geopandas>=0.12.2


### 2.3 Import necessary Python packages

In [18]:
import os
import glob
import math

from numba import jit, njit
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.ndimage import convolve


## 3. Configuration

In [ ]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# population rasters
population_dir = "/Users/d3y010/projects/population/data/population_conus_total_ssp2_2020_1km.asc"


## 4. Generate population rasters

### 4.1 Functions

In [20]:
def arr_to_ascii(arr, r_ascii, xll=-180, yll=-90, cellsize=0.25, nodata=-999):
    """
    Convert a numpy array to an ASCII raster.
    :@param arr:            2D array
    :@param r_ascii:        Full path to outfile with extension
    :@param xll:            Longitude coordinate for lower left corner
    :@param yll:            Latitude coordinate for lower left corner
    :@param cellsize:       Cell size in geographic degrees
    :@param nodata:         Value representing NODATA
    """

    # get number of rows and columns of array
    nrows = arr.shape[0]
    ncols = arr.shape[1]

    # create ASCII raster file
    with open(r_ascii, 'w') as rast:

        # write header
        rast.write('ncols {}\n'.format(ncols))
        rast.write('nrows {}\n'.format(nrows))
        rast.write('xllcorner {}\n'.format(xll))
        rast.write('yllcorner {}\n'.format(yll))
        rast.write('cellsize {}\n'.format(cellsize))
        rast.write('nodata_value {}\n'.format(nodata))

        # write array
        np.savetxt(rast, arr, fmt='%.15g')
        


# @jit(nopython=True)
def create_buffer(radius):  #radius given in #cells
    
    b = []
    for i in range(radius+1):
        
        for j in range(radius+1):
            
            if math.sqrt(i**2 + j**2) <= radius:
                
                if i==0 and j==0:
                    b.extend([(i,j)])
                elif i==0:
                    b.extend([(i,j),(i,-j)])
                elif j==0:
                    b.extend([(i,j),(-i,j)])
                else:
                    b.extend([(i,j),(-i,j),(i,-j),(-i,-j)])
                    
    return b


# @jit(nopython=True)
def check_unsuitable_cells(unsuitable_cells, unsuitable_shapes, threshold):
    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]

        while len(to_check) > 0:
            cell = to_check.pop()
            for i, j in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                if (cell[0]+i, cell[1]+j) in unsuitable_cells:
                    to_check.append((cell[0]+i, cell[1]+j))
                    this_shape.append((cell[0]+i, cell[1]+j))
                    unsuitable_cells.remove((cell[0]+i, cell[1]+j))

        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    return unsuitable_cells, unsuitable_shapes


def main(input_raster, out_dir, scenario, year=2100, buffer_km=40):
 
    buffer_relative = create_buffer(buffer_km)        #25 mile (40 km) buffer
    buffer_relative.reverse()                   #Look at biggest buffers first

    unbuffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}.asc")
    buffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_buff25mi.asc")
    nuclear_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_nuclear.asc")

    headers = []
    in_raster_data = []
    unsuitable_cells = []
    unsuitable_shapes = []
    buffered_shapes = []

    print('Loading input raster\n')
    inf = open(input_raster,'r')
    currentline = inf.readline()

    nodata_val = '-999'

    while currentline.split()[0] != nodata_val:
        headers.append(currentline)
        currentline = inf.readline()
    while currentline:
        in_raster_data.append([int(x) for x in currentline.split()])
        currentline = inf.readline()
    inf.close()

    print( 'Producing unbuffered file\n')
    #produce unbuffered file
    f = open(unbuffered_file,'w')
    for line in headers:
        f.write(line)
        
    for rowi in range(len(in_raster_data)):
        for columni in range(len(in_raster_data[rowi])):
            cell = in_raster_data[rowi][columni]
            threshold = 772
                
            if cell > threshold:
                f.write('1 ')
                unsuitable_cells.append((rowi,columni))
                
            elif cell == -9999:
                f.write('1 ')
            else:
                f.write('0 ')
        f.write('\n')
    f.close()

    print( 'Producing buffered file\n')
    print( '...identifying unsuitable blocks')
            
    threshold = 64.75

    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]
        
        while len(to_check) > 0:
            cell = to_check.pop()
            if (cell[0],cell[1]-1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]-1))
                this_shape.append((cell[0],cell[1]-1))
                unsuitable_cells.remove((cell[0],cell[1]-1))
            if (cell[0],cell[1]+1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]+1))
                this_shape.append((cell[0],cell[1]+1))
                unsuitable_cells.remove((cell[0],cell[1]+1))
            if (cell[0]-1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]-1,cell[1]))
                this_shape.append((cell[0]-1,cell[1]))
                unsuitable_cells.remove((cell[0]-1,cell[1]))
            if (cell[0]+1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]+1,cell[1]))
                this_shape.append((cell[0]+1,cell[1]))
                unsuitable_cells.remove((cell[0]+1,cell[1]))
                
        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    unsuitable_cells = this_shape = None     #free up memory

    print( '...buffering')
    buffered_shapes = set(buffered_shapes)
    
    for cell in unsuitable_shapes:
        
        for offset in buffer_relative:
            buffered_shapes.add((cell[0]+offset[0], cell[1]+offset[1]))


    print( '...printing result\n')
    with open(buffered_file,'w') as f:
        for line in headers:
            f.write(line)
        for rowi in range(len(in_raster_data)):
            for columni in range(len(in_raster_data[rowi])):
                if (rowi,columni) in buffered_shapes:
                    f.write('1 ')
                elif in_raster_data[rowi][columni] == -9999:
                    f.write('1 ')
                else:
                    f.write('0 ')
            f.write('\n')
    

    print('Producing nuclear file...')
    buffer_relative=[]
    for i in range(40+1):
        buffer_relative.append(create_buffer(i))
    cell_area = 1
        
    nrows = len(in_raster_data)
    ncolumns = len(in_raster_data[0])
    
    f = open(nuclear_file,'w')
    for line in headers:
        f.write(line)
    for rowi in range(nrows):

        for columni in range(ncolumns):
            suitable = True
            break_flag = False
            for radius in buffer_relative:
                if not break_flag:
                    totalpop = 0
                    totalarea = 0

                    for offset in radius:
                        new_row = rowi + offset[0]
                        new_column = columni + offset[1]
                        if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
                           in_raster_data[new_row][new_column] != -9999:
                            totalpop += in_raster_data[new_row][new_column]
                            totalarea += cell_area

                    if totalarea != 0:
                        density = float(totalpop)/totalarea
                        if density > 193:
                            suitable = False
                            break_flag = True
                        elif density < 30:
                            break_flag = True
                    else:
                        suitable = False
                        break_flag = True
            if suitable:
                f.write('0 ')
            else:
                f.write('1 ')
        f.write('\n')
    f.close()
    
    

### 4.2 This code generates the population files on a year by year basis

In [ ]:
%%time

outcome = main(f_asc, 
                 out_dir=os.path.dirname(f_asc), 
                 scenario="ssp2", 
                 year=2020)


nuclear_file, headers, nrows, ncolumns, buffer_relative, in_raster_data = outcome


In [9]:
import numpy as np
import rasterio
from scipy.ndimage import convolve

population_per_kilometer = 772
buffer_in_km = 40

# raster for gridded 1 kilometer population over the contiguous united states
population_raster_file = "/Users/d3y010/projects/population/data/population_conus_total_ssp2_2020_1km.tif"

# output raster files
dense_population_raster_file = "/Users/d3y010/projects/population/data/dense_population_area.tif"
buffered_dense_population_raster_file = "/Users/d3y010/projects/population/data/buffered_dense_population_area.tif"


# read in raster file to a numpy array
with rasterio.open(population_raster_file) as src:
    metadata = src.meta.copy()
    population_array = src.read(1)

# determine population density 
dense_population_area_array = np.where(population_array >= population_per_kilometer, 1, 0).astype(np.float32)

# write dense population raster file
with rasterio.open(dense_population_raster_file, 'w', **metadata) as dst:
    dst.write(dense_population_area_array, 1)
    


In [23]:
# buffer each gridcell in the dense population area array by the kernel size
from skimage.morphology import dilation, square, disk


In [26]:
%%time

kernel_size = int(buffer_in_km / 2)
# selem = square(kernel_size)
selem = disk(kernel_size)

# buffered_dense_population_array = dilation(dense_population_area_array, selem)

def count_zeros(arr):
    return np.sum(arr == 0)

conv_result = convolve(dense_population_area_array, selem, mode='constant', cval=0, origin=0, function=count_zeros)

dense_population_area_array[conv_result >= 60] = 0
dense_population_area_array[conv_result < 60] = 1


TypeError: convolve() got an unexpected keyword argument 'function'

In [25]:
buffered_dense_population_array = np.where(buffered_dense_population_array >= 1, 1, 0)

# write dense population raster file
with rasterio.open(buffered_dense_population_raster_file, 'w', **metadata) as dst:
    dst.write(buffered_dense_population_array, 1)


## New code

In [2]:
population_per_kilometer = 772
buffer_in_km = 40

# raster for gridded 1 kilometer population over the contiguous united states
population_raster_file = "/Users/d3y010/projects/population/data/population_conus_total_ssp2_2020_1km.tif"

# output raster files
dense_population_raster_file = "/Users/d3y010/projects/population/data/dense_population_area.tif"
buffered_dense_population_raster_file = "/Users/d3y010/projects/population/data/buffered_dense_population_area.tif"


# read in raster file to a numpy array
with rasterio.open(population_raster_file) as src:
    metadata = src.meta.copy()
    population_array = src.read(1)

# determine population density 
dense_population_area_array = np.where(population_array >= population_per_kilometer, 1, 0).astype(np.float32)

# write dense population raster file
with rasterio.open(dense_population_raster_file, 'w', **metadata) as dst:
    dst.write(dense_population_area_array, 1)
    



In [8]:
from scipy.ndimage import convolve

kernel_size = int(buffer_in_km / 2)
kernel = np.ones((kernel_size, kernel_size), dtype=int)

convolved_array = convolve(dense_population_area_array, kernel, mode='constant', cval=0)

# Set all elements in the array equal to 1 where the kernel equals 1
buffered_dense_population_array = np.where(convolved_array >= 1, 1, 0)

# write dense population raster file
with rasterio.open(buffered_dense_population_raster_file, 'w', **metadata) as dst:
    dst.write(buffered_dense_population_array, 1)


In [9]:
f = "/Users/d3y010/projects/population/data/population_conus_total_ssp2_2020_1km.tif"
f_asc = "/Users/d3y010/projects/population/data/population_conus_total_ssp2_2020_1km.asc"


In [13]:
def arr_to_ascii(arr, r_ascii, xll=-180, yll=-90, cellsize=0.25, nodata=-999):
    """
    Convert a numpy array to an ASCII raster.
    :@param arr:            2D array
    :@param r_ascii:        Full path to outfile with extension
    :@param xll:            Longitude coordinate for lower left corner
    :@param yll:            Latitude coordinate for lower left corner
    :@param cellsize:       Cell size in geographic degrees
    :@param nodata:         Value representing NODATA
    """

    # get number of rows and columns of array
    nrows = arr.shape[0]
    ncols = arr.shape[1]

    # create ASCII raster file
    with open(r_ascii, 'w') as rast:

        # write header
        rast.write('ncols {}\n'.format(ncols))
        rast.write('nrows {}\n'.format(nrows))
        rast.write('xllcorner {}\n'.format(xll))
        rast.write('yllcorner {}\n'.format(yll))
        rast.write('cellsize {}\n'.format(cellsize))
        rast.write('nodata_value {}\n'.format(nodata))

        # write array
        np.savetxt(rast, arr, fmt='%.15g')
        


@jit(nopython=True)
def create_buffer(radius):  #radius given in #cells
    
    b = []
    for i in range(radius+1):
        
        for j in range(radius+1):
            
            if math.sqrt(i**2 + j**2) <= radius:
                
                if i==0 and j==0:
                    b.extend([(i,j)])
                elif i==0:
                    b.extend([(i,j),(i,-j)])
                elif j==0:
                    b.extend([(i,j),(-i,j)])
                else:
                    b.extend([(i,j),(-i,j),(i,-j),(-i,-j)])
                    
    return b


@jit(nopython=True)
def check_unsuitable_cells(unsuitable_cells, unsuitable_shapes, threshold):
    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]

        while len(to_check) > 0:
            cell = to_check.pop()
            for i, j in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                if (cell[0]+i, cell[1]+j) in unsuitable_cells:
                    to_check.append((cell[0]+i, cell[1]+j))
                    this_shape.append((cell[0]+i, cell[1]+j))
                    unsuitable_cells.remove((cell[0]+i, cell[1]+j))

        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    return unsuitable_cells, unsuitable_shapes


def main(input_raster, out_dir, scenario, year=2100, buffer_km=40):
 
    buffer_relative = create_buffer(buffer_km)        #25 mile (40 km) buffer
    buffer_relative.reverse()                   #Look at biggest buffers first

    unbuffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}.asc")
    buffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_buff25mi.asc")
    nuclear_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_nuclear.asc")

    headers = []
    in_raster_data = []
    unsuitable_cells = []
    unsuitable_shapes = []
    buffered_shapes = []

    print('Loading input raster\n')
    inf = open(input_raster,'r')
    currentline = inf.readline()

    nodata_val = '-999'

    while currentline.split()[0] != nodata_val:
        headers.append(currentline)
        currentline = inf.readline()
    while currentline:
        in_raster_data.append([int(x) for x in currentline.split()])
        currentline = inf.readline()
    inf.close()

    print( 'Producing unbuffered file\n')
    #produce unbuffered file
    f = open(unbuffered_file,'w')
    for line in headers:
        f.write(line)
        
    for rowi in range(len(in_raster_data)):
        for columni in range(len(in_raster_data[rowi])):
            cell = in_raster_data[rowi][columni]
            threshold = 772
                
            if cell > threshold:
                f.write('1 ')
                unsuitable_cells.append((rowi,columni))
                
            elif cell == -9999:
                f.write('1 ')
            else:
                f.write('0 ')
        f.write('\n')
    f.close()

    print( 'Producing buffered file\n')
    print( '...identifying unsuitable blocks')

    threshold = 64.75

    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]
        
        while len(to_check) > 0:
            cell = to_check.pop()
            if (cell[0],cell[1]-1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]-1))
                this_shape.append((cell[0],cell[1]-1))
                unsuitable_cells.remove((cell[0],cell[1]-1))
            if (cell[0],cell[1]+1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]+1))
                this_shape.append((cell[0],cell[1]+1))
                unsuitable_cells.remove((cell[0],cell[1]+1))
            if (cell[0]-1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]-1,cell[1]))
                this_shape.append((cell[0]-1,cell[1]))
                unsuitable_cells.remove((cell[0]-1,cell[1]))
            if (cell[0]+1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]+1,cell[1]))
                this_shape.append((cell[0]+1,cell[1]))
                unsuitable_cells.remove((cell[0]+1,cell[1]))
                
        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    unsuitable_cells = this_shape = None     #free up memory

    print( '...buffering')
    buffered_shapes = set(buffered_shapes)
    
    for cell in unsuitable_shapes:
        
        for offset in buffer_relative:
            buffered_shapes.add((cell[0]+offset[0], cell[1]+offset[1]))


#     print( '...printing result\n')
#     with open(buffered_file,'w') as f:
#         for line in headers:
#             f.write(line)
#         for rowi in range(len(in_raster_data)):
#             for columni in range(len(in_raster_data[rowi])):
#                 if (rowi,columni) in buffered_shapes:
#                     f.write('1 ')
#                 elif in_raster_data[rowi][columni] == -9999:
#                     f.write('1 ')
#                 else:
#                     f.write('0 ')
#             f.write('\n')
    

#     print('Producing nuclear file...')
#     buffer_relative=[]
#     for i in range(40+1):
#         buffer_relative.append(create_buffer(i))
#     cell_area = 1
        
#     nrows = len(in_raster_data)
#     ncolumns = len(in_raster_data[0])
    
#     f = open(nuclear_file,'w')
#     for line in headers:
#         f.write(line)
#     for rowi in range(nrows):

#         for columni in range(ncolumns):
#             suitable = True
#             break_flag = False
#             for radius in buffer_relative:
#                 if not break_flag:
#                     totalpop = 0
#                     totalarea = 0

#                     for offset in radius:
#                         new_row = rowi + offset[0]
#                         new_column = columni + offset[1]
#                         if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
#                            in_raster_data[new_row][new_column] != -9999:
#                             totalpop += in_raster_data[new_row][new_column]
#                             totalarea += cell_area

#                     if totalarea != 0:
#                         density = float(totalpop)/totalarea
#                         if density > 193:
#                             suitable = False
#                             break_flag = True
#                         elif density < 30:
#                             break_flag = True
#                     else:
#                         suitable = False
#                         break_flag = True
#             if suitable:
#                 f.write('0 ')
#             else:
#                 f.write('1 ')
#         f.write('\n')
#     f.close()
    
    



In [5]:
def arr_to_ascii(arr, r_ascii, xll=-180, yll=-90, cellsize=0.25, nodata=-999):
    """
    Convert a numpy array to an ASCII raster.
    :@param arr:            2D array
    :@param r_ascii:        Full path to outfile with extension
    :@param xll:            Longitude coordinate for lower left corner
    :@param yll:            Latitude coordinate for lower left corner
    :@param cellsize:       Cell size in geographic degrees
    :@param nodata:         Value representing NODATA
    """

    # get number of rows and columns of array
    nrows = arr.shape[0]
    ncols = arr.shape[1]

    # create ASCII raster file
    with open(r_ascii, 'w') as rast:

        # write header
        rast.write('ncols {}\n'.format(ncols))
        rast.write('nrows {}\n'.format(nrows))
        rast.write('xllcorner {}\n'.format(xll))
        rast.write('yllcorner {}\n'.format(yll))
        rast.write('cellsize {}\n'.format(cellsize))
        rast.write('nodata_value {}\n'.format(nodata))

        # write array
        np.savetxt(rast, arr, fmt='%.15g')
        


def create_buffer(radius):  #radius given in #cells
    
    b = []
    for i in range(radius+1):
        
        for j in range(radius+1):
            
            if math.sqrt(i**2 + j**2) <= radius:
                
                if i==0 and j==0:
                    b.extend([(i,j)])
                elif i==0:
                    b.extend([(i,j),(i,-j)])
                elif j==0:
                    b.extend([(i,j),(-i,j)])
                else:
                    b.extend([(i,j),(-i,j),(i,-j),(-i,-j)])
                    
    return b


@jit(nopython=True)
def check_suitability(rowi, columni, in_raster_data, buffer_relative, nrows, ncolumns):
    suitable = True
    break_flag = False
    for radius in buffer_relative:
        if not break_flag:
            totalpop = 0
            totalarea = 0

            for offset in radius:
                new_row = rowi + offset[0]
                new_column = columni + offset[1]
                if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
                   in_raster_data[new_row][new_column] != -9999:
                    totalpop += in_raster_data[new_row][new_column]
                    totalarea += cell_area

            if totalarea != 0:
                density = float(totalpop)/totalarea
                if density > 193:
                    suitable = False
                    break_flag = True
                elif density < 30:
                    break_flag = True
            else:
                suitable = False
                break_flag = True
    return suitable


@jit(nopython=True)
def check_unsuitable_cells(unsuitable_cells, threshold):
    unsuitable_shapes = List()
    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]

        while len(to_check) > 0:
            cell = to_check.pop()
            for i, j in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                if (cell[0]+i, cell[1]+j) in unsuitable_cells:
                    to_check.append((cell[0]+i, cell[1]+j))
                    this_shape.append((cell[0]+i, cell[1]+j))
                    unsuitable_cells.remove((cell[0]+i, cell[1]+j))

        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    return unsuitable_cells, unsuitable_shapes


def main(input_raster, out_dir, scenario, year=2100, buffer_km=40):
 
    buffer_relative = create_buffer(buffer_km)        #25 mile (40 km) buffer
    buffer_relative.reverse()                   #Look at biggest buffers first

    unbuffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}.asc")
    buffered_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_buff25mi.asc")
    nuclear_file = os.path.join(out_dir, f"cerf_densely_populated_{scenario}_{year}_nuclear.asc")

    headers = []
    in_raster_data = []
    unsuitable_cells = List()
    unsuitable_shapes = List()
    buffered_shapes = List()

    print('Loading input raster\n')
    inf = open(input_raster,'r')
    currentline = inf.readline()

    nodata_val = '-999'

    while currentline.split()[0] != nodata_val:
        headers.append(currentline)
        currentline = inf.readline()
    while currentline:
        in_raster_data.append([int(x) for x in currentline.split()])
        currentline = inf.readline()
    inf.close()

    print( 'Producing unbuffered file\n')
    #produce unbuffered file
    f = open(unbuffered_file,'w')
    for line in headers:
        f.write(line)
        
    for rowi in range(len(in_raster_data)):
        for columni in range(len(in_raster_data[rowi])):
            cell = in_raster_data[rowi][columni]
            threshold = 772
                
            if cell > threshold:
                f.write('1 ')
                unsuitable_cells.append((rowi,columni))
                
            elif cell == -9999:
                f.write('1 ')
            else:
                f.write('0 ')
        f.write('\n')
    f.close()

    print( 'Producing buffered file\n')
    print( '...identifying unsuitable blocks')
    threshold = 64.75
    
    unsuitable_cells, unsuitable_shapes = check_unsuitable_cells(unsuitable_cells, 
                                                                 threshold)

    print( '...buffering')
    buffered_shapes = set(buffered_shapes)
    
    for cell in unsuitable_shapes:
        
        for offset in buffer_relative:
            buffered_shapes.add((cell[0]+offset[0], cell[1]+offset[1]))


    print( '...printing result\n')
    with open(buffered_file,'w') as f:
        for line in headers:
            f.write(line)
        for rowi in range(len(in_raster_data)):
            for columni in range(len(in_raster_data[rowi])):
                if (rowi,columni) in buffered_shapes:
                    f.write('1 ')
                elif in_raster_data[rowi][columni] == -9999:
                    f.write('1 ')
                else:
                    f.write('0 ')
            f.write('\n')
    
    print('Producing nuclear file...')
    buffer_relative=[]
    for i in range(40+1):
        buffer_relative.append(create_buffer(i))
    cell_area = 1
        
    nrows = len(in_raster_data)
    ncolumns = len(in_raster_data[0])
    
    return nuclear_file, headers, nrows, ncolumns, buffer_relative, in_raster_data
    
#     with open(nuclear_file,'w') as f:
#         for line in headers:
#             f.write(line)

#         # for each grid cell row[0], col[0] ... n
#         for rowi, columni in itertools.product(range(nrows), range(ncolumns)):

#             suitable = True
#             break_flag = False

#             for radius in buffer_relative:
#                 if not break_flag:
#                     totalpop = 0
#                     totalarea = 0

#                     for offset in radius:
#                         new_row = rowi + offset[0]
#                         new_column = columni + offset[1]

#                         if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
#                            in_raster_data[new_row][new_column] != -9999:
#                             totalpop += in_raster_data[new_row][new_column]
#                             totalarea += cell_area

#                     if totalarea != 0:
#                         density = float(totalpop)/totalarea

#                         if density > 193:
#                             suitable = False
#                             break_flag = True
#                         elif density < 30:
#                             break_flag = True
#                     else:
#                         suitable = False
#                         break_flag = True
#             if suitable:
#                 f.write('0 ')
#             else:
#                 f.write('1 ')
#         f.write('\n')
   



In [14]:
nodata = -999

with rasterio.open(f) as src:

    arr = src.read(1)

    arr = np.where(arr == src.nodata, nodata, arr)

    # round up to whole numbers
    arr = np.around(arr, 0).astype(np.int64)

    arr_to_ascii(arr, f_asc, xll=src.bounds.left, yll=src.bounds.bottom, cellsize=1000, nodata=nodata)



In [7]:
import math

In [8]:
%%time

outcome = main(f_asc, 
     out_dir=os.path.dirname(f_asc), 
     scenario="ssp2", 
     year=2020)


nuclear_file, headers, nrows, ncolumns, buffer_relative, in_raster_data = outcome


Loading input raster

Producing unbuffered file

Producing buffered file

...identifying unsuitable blocks
...buffering
...printing result

Producing nuclear file...
CPU times: user 1min 54s, sys: 951 ms, total: 1min 55s
Wall time: 1min 56s


In [26]:
nuclear_file

'/Users/d3y010/projects/population/data/cerf_densely_populated_ssp2_2020_nuclear.asc'

In [27]:
headers

['ncols 4815\n',
 'nrows 3104\n',
 'xllcorner -2456207.6475\n',
 'yllcorner -1437505.3457\n',
 'cellsize 1000\n',
 'nodata_value -999\n']

In [29]:
nrows, ncolumns

(3104, 4815)

In [31]:
len(buffer_relative)

41

In [34]:
len(in_raster_data)

3104

In [36]:
len(in_raster_data[0])

4815

In [39]:
with open(nuclear_file,'w') as f:
    for line in headers:
        f.write(line)
        
    # for each grid cell row[0], col[0] ... n
    for rowi, columni in itertools.product(range(nrows), range(ncolumns)):

        suitable = True
        break_flag = False

        for radius in buffer_relative:
            if not break_flag:
                totalpop = 0
                totalarea = 0

                for offset in radius:
                    new_row = rowi + offset[0]
                    new_column = columni + offset[1]

                    if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
                       in_raster_data[new_row][new_column] != -9999:
                        totalpop += in_raster_data[new_row][new_column]
                        totalarea += cell_area

                if totalarea != 0:
                    density = float(totalpop)/totalarea

                    if density > 193:
                        suitable = False
                        break_flag = True
                    elif density < 30:
                        break_flag = True
                else:
                    suitable = False
                    break_flag = True
        if suitable:
            f.write('0 ')
        else:
            f.write('1 ')
    f.write('\n')

NameError: name 'cell_area' is not defined

## 3. Configuration

In [2]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source data directory
source_dir = os.path.join(gridcerf_dir, "source", "water")

# temporary output raster for processing
temp_output_raster = os.path.join(source_dir, "temporary_raster.tif")

# generate a list of all common exclusion files
common_raster_list = glob.glob(os.path.join(common_dir, "*.tif"))

# source NHD shapefile
nhd_shapefile = os.path.join(source_dir, "surface_water_flow_nhdplus_v2_erom_eispc_v2", "ez_gis.surface_water_flow_nhdplus_v2_erom_eispc_v2.shp")

# bins for minimum mean annual flow requirements where the key is the target 
#  file name and the value is the threshold in MGD
mgd_list = [2, 10, 25, 35, 40, 55, 70, 75, 95, 110, 120, 135]


## 4. Generate wind suitability rasters

### 4.1 Functions to build suitability

In [3]:
def preprocess_nhd_data(nhd_shapefile: str,
                        template_raster: str) -> gpd.GeoDataFrame:
    """Preprocess NHD flowlines to convert to millions gallons per day (MGD) and prepare
    a rasterization field.
    
    """
    
    # get target coordinate reference system from template raster
    with rasterio.open(template_raster) as src:
        target_crs = src.crs
    
    # only keep gallons per minute flow and geometry and reproject
    gdf = gpd.read_file(nhd_shapefile)[['q_gpm', 'geometry']].to_crs(target_crs)

    # convert to millions gallons per day
    gdf['mgd'] = (gdf['q_gpm'] / 1000000) * 60 * 24

    # drop gpm field
    gdf.drop(columns=['q_gpm'], inplace=True)

    # set raster value
    gdf['value'] = 0
    
    return gdf
    

### 4.2 Generate available water suitability rasters

In [8]:
# preprocess NHD flowlines for rasterization to MGD thresholds
print("Preprocessing NHD data...")
gdf = preprocess_nhd_data(nhd_shapefile=nhd_shapefile,
                          template_raster=template_raster)

# create buffered flowlines matching the flow requirement
for i in mgd_list:
    
    print(f"Processing flow threshold of:  {i} MGD")
    
    # construct a file basename
    basename = f"gridcerf_nhd2plus_surfaceflow_greaterthan{i}mgd_buffer20km"
    
    # extract the flowlines that support the minimum flow requirement
    gdx = gdf.loc[gdf['mgd'] >= i].copy()

    # buffer by 20 km (20000 meters)
    gdx['geometry'] = gdx.buffer(20000)
    
    # construct temporary shapefile output file path
    output_shp = os.path.join(source_dir, f"{basename}.shp")
    
    # write output shapefile
    gdx[['value', 'geometry']].to_file(output_shp)
    
    # construct the water availability raster output raster name
    output_raster = os.path.join(technology_specific_dir, f"{basename}.tif")

    # construct the GDAL raster command
    gdal_rasterize_cmd = f"gdal_rasterize -l {basename} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {output_shp} {output_raster}"
    
    # execute the GDAL command via the system terminal
    os.system(gdal_rasterize_cmd)


Preprocessing NHD data...
Processing flow threshold of:  2 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  10 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  25 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  35 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  40 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  55 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  70 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  75 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  95 MGD
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing flow threshold of:  110 MGD
0...10...20...30...40...50...60...70...80...9